In [80]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

C:\Users\katha\AppData\Local\Temp\ipykernel_56652\3777615979.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


# Lab | Natural Language Processing
### SMS: SPAM or HAM

### Let's prepare the environment

In [103]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

- Read Data for the Fraudulent Email Kaggle Challenge
- Reduce the training set to speead up development. 

In [82]:
## Read Data for the Fraudulent Email Kaggle Challenge
data = pd.read_csv('../data/kg_train.csv',encoding='latin-1')

# Reduce the training set to speed up development. 
# Modify for final system
data = data.head(1000)
print(data.shape)
data.fillna("",inplace=True)

(1000, 2)


### Let's divide the training and test set into two partitions

In [83]:
#x = data["text"]
#y = data["label"]
data_train, data_val = train_test_split(data, test_size=0.3, random_state=42, stratify=data['label'])

# Print the results to verify the split
print("Training DataFrame:")
print(data_train.head())

print("\nValidation DataFrame:")
print(data_val.head())

Training DataFrame:
                                                  text  label
381  Mills Cheryl D <MillsCD@state.gov>Sunday Janua...      0
428  H <hrod17@clintonemail.com >Saturday January 2...      0
849  DEAR,     MY NAME IS MR MR Ken Edward,A former...      1
252  Dear Sir, I am Engr. Victor Chigoziem with the...      1
380  Hello,This is Dr.Clive Whittaker. I work for F...      1

Validation DataFrame:
                                                  text  label
289  No=2E 5667 Allen Avenue=2CVictoria Garden City...      1
984  He's in cairo. Saying after 2:30 is ideal. Sec...      0
785  Robert Pape is a top strategic thinker from U ...      0
123  Inside CGI.. More later.This is written from m...      0
615  DEAR RESPECTFUL ONE, I AM MISS ALICIA MADOUH  ...      1


## Data Preprocessing

In [84]:
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
from bs4 import BeautifulSoup, Comment
print(string.punctuation)
print(stopwords.words("english")[100:110])
from nltk.stem.snowball import SnowballStemmer
snowball = SnowballStemmer('english')

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
['here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each']


## Now, we have to clean the html code removing words

- First we remove inline JavaScript/CSS
- Then we remove html comments. This has to be done before removing regular tags since comments can contain '>' characters
- Next we can remove the remaining tags

In [85]:
def clean_html(html):

    # parse html content
    soup = BeautifulSoup(html, "html.parser")

    for data in soup(['style', 'script', 'code', 'a']):
        # Remove tags
        data.decompose()

    # Optionally handle comments separately
    for element in soup(text=lambda text: isinstance(text, Comment)):
        element.extract()

    # Return cleaned text
    return ' '.join(soup.stripped_strings)

# Sample application to x_train and x_test
data_train["preprocessed_text"] = data_train["text"].apply(clean_html)
data_val["preprocessed_text"] = data_val["text"].apply(clean_html)

print(data_train.head())
print(data_val.head())


C:\Users\katha\AppData\Local\Temp\ipykernel_56652\1852950438.py:4: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(html, "html.parser")


                                                  text  label  \
381  Mills Cheryl D <MillsCD@state.gov>Sunday Janua...      0   
428  H <hrod17@clintonemail.com >Saturday January 2...      0   
849  DEAR,     MY NAME IS MR MR Ken Edward,A former...      1   
252  Dear Sir, I am Engr. Victor Chigoziem with the...      1   
380  Hello,This is Dr.Clive Whittaker. I work for F...      1   

                                     preprocessed_text  
381  Mills Cheryl D Sunday January 31 2010 12:17 PM...  
428  H Saturday January 23 2010 4:09 PM'sbwhoeopRe:...  
849  DEAR,     MY NAME IS MR MR Ken Edward,A former...  
252  Dear Sir, I am Engr. Victor Chigoziem with the...  
380  Hello,This is Dr.Clive Whittaker. I work for F...  
                                                  text  label  \
289  No=2E 5667 Allen Avenue=2CVictoria Garden City...      1   
984  He's in cairo. Saying after 2:30 is ideal. Sec...      0   
785  Robert Pape is a top strategic thinker from U ...      0   
123  In

C:\Users\katha\AppData\Local\Temp\ipykernel_56652\1852950438.py:4: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(html, "html.parser")
C:\Users\katha\AppData\Local\Temp\ipykernel_56652\1852950438.py:4: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  soup = BeautifulSoup(html, "html.parser")


- Remove all the special characters
    
- Remove numbers
    
- Remove all single characters
 
- Remove single characters from the start

- Substitute multiple spaces with single space

- Remove prefixed 'b'

- Convert to Lowercase

In [90]:
def clean_text(text):
    # Remove special characters
    text = re.sub(r'[^A-Za-z\s]', '', text)

    # Remove numbers
    text = re.sub(r'\d+', '', text)

    # Remove all single characters except 'I' when it stands alone
    text = re.sub(r'\b(?!\bI\b)\w\b', '', text)
    
    # Remove single characters from start
    text = re.sub(r'\b\w\b', '', text)
    
    # Substitute multiple spaces with single space
    text = re.sub(r'\s+', ' ', text)

    # Remove prefixed 'b' 
    text = text.replace("b'", "").replace("b\"", "")

    # Convert to lowercase
    text = text.lower().strip()

    return text

# Clean text
data_train["preprocessed_text"] = data_train["preprocessed_text"].apply(clean_text)
data_val["preprocessed_text"] = data_val["preprocessed_text"].apply(clean_text)

print(data_train.head())
print(data_val.head())

                                                  text  label  \
381  Mills Cheryl D <MillsCD@state.gov>Sunday Janua...      0   
428  H <hrod17@clintonemail.com >Saturday January 2...      0   
849  DEAR,     MY NAME IS MR MR Ken Edward,A former...      1   
252  Dear Sir, I am Engr. Victor Chigoziem with the...      1   
380  Hello,This is Dr.Clive Whittaker. I work for F...      1   

                                     preprocessed_text  
381  mill cheryl sunday january pmtravel scheduleca...  
428  saturday january pmsbwhoeopre fyi foreign nati...  
849  dear name mr mr ken edwarda former government ...  
252  dear sir engr victor chigoziem engineering sto...  
380  hellothis drclive whittaker work fidelity inve...  
                                                  text  label  \
289  No=2E 5667 Allen Avenue=2CVictoria Garden City...      1   
984  He's in cairo. Saying after 2:30 is ideal. Sec...      0   
785  Robert Pape is a top strategic thinker from U ...      0   
123  In

## Now let's work on removing stopwords
Remove the stopwords.

In [91]:
# Get the list of stopwords
stop_words = set(stopwords.words('english'))

# Create a pattern that matches any of the stopwords
stopwords_pattern = r'\b(?:' + '|'.join(stop_words) + r')\b'

def remove_stopwords(text):
    # Use the regex pattern to remove stopwords
    cleaned_text = re.sub(stopwords_pattern, '', text)
    # Remove any extra spaces created due to stopword removal
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()
    return cleaned_text

# Apply the stopword removal function to the cleaned datasets
data_train["preprocessed_text"] = data_train["preprocessed_text"].apply(remove_stopwords)
data_val["preprocessed_text"] = data_val["preprocessed_text"].apply(remove_stopwords)

print(data_train.head())
print(data_val.head())

                                                  text  label  \
381  Mills Cheryl D <MillsCD@state.gov>Sunday Janua...      0   
428  H <hrod17@clintonemail.com >Saturday January 2...      0   
849  DEAR,     MY NAME IS MR MR Ken Edward,A former...      1   
252  Dear Sir, I am Engr. Victor Chigoziem with the...      1   
380  Hello,This is Dr.Clive Whittaker. I work for F...      1   

                                     preprocessed_text  
381  mill cheryl sunday january pmtravel scheduleca...  
428  saturday january pmsbwhoeopre fyi foreign nati...  
849  dear name mr mr ken edwarda former government ...  
252  dear sir engr victor chigoziem engineering sto...  
380  hellothis drclive whittaker work fidelity inve...  
                                                  text  label  \
289  No=2E 5667 Allen Avenue=2CVictoria Garden City...      1   
984  He's in cairo. Saying after 2:30 is ideal. Sec...      0   
785  Robert Pape is a top strategic thinker from U ...      0   
123  In

## Tame Your Text with Lemmatization
Break sentences into words, then use lemmatization to reduce them to their base form (e.g., "running" becomes "run"). See how this creates cleaner data for analysis!

In [92]:
# Initialize Lemmatizer
lemmatizer = WordNetLemmatizer()

def lemmatize_text(text):
    words = text.split()
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
    return ' '.join(lemmatized_words)

# Apply lemmatization to the text
data_train["preprocessed_text"] = data_train["preprocessed_text"].apply(lemmatize_text)
data_val["preprocessed_text"] = data_val["preprocessed_text"].apply(lemmatize_text)

print(data_train.head())
print(data_val.head())

                                                  text  label  \
381  Mills Cheryl D <MillsCD@state.gov>Sunday Janua...      0   
428  H <hrod17@clintonemail.com >Saturday January 2...      0   
849  DEAR,     MY NAME IS MR MR Ken Edward,A former...      1   
252  Dear Sir, I am Engr. Victor Chigoziem with the...      1   
380  Hello,This is Dr.Clive Whittaker. I work for F...      1   

                                     preprocessed_text  
381  mill cheryl sunday january pmtravel scheduleca...  
428  saturday january pmsbwhoeopre fyi foreign nati...  
849  dear name mr mr ken edwarda former government ...  
252  dear sir engr victor chigoziem engineering sto...  
380  hellothis drclive whittaker work fidelity inve...  
                                                  text  label  \
289  No=2E 5667 Allen Avenue=2CVictoria Garden City...      1   
984  He's in cairo. Saying after 2:30 is ideal. Sec...      0   
785  Robert Pape is a top strategic thinker from U ...      0   
123  In

## Bag Of Words
Let's get the 10 top words in ham and spam messages (**EXPLORATORY DATA ANALYSIS**)

In [93]:
from collections import Counter

# Separate ham (y=0) and spam (y=1) messages based on labels
ham_messages = data_train[data_train["label"] == 0]["preprocessed_text"]
spam_messages = data_train[data_train["label"] == 1]["preprocessed_text"]

# Count the words in the ham and spam lists
ham_word_counts = Counter(" ".join(ham_messages).split())
spam_word_counts = Counter(" ".join(spam_messages).split())

# Display top 10 words 
top_10_ham_words = ham_word_counts.most_common(10)
top_10_spam_words = spam_word_counts.most_common(10)

print("Top 10 words in ham messages:")
for word, freq in top_10_ham_words:
    print(f"{word}: {freq}")

print("\nTop 10 words in spam messages:")
for word, freq in top_10_spam_words:
    print(f"{word}: {freq}")


Top 10 words in ham messages:
president: 83
would: 80
percent: 76
call: 66
state: 65
american: 60
work: 58
obama: 57
pm: 57
mr: 56

Top 10 words in spam messages:
money: 594
account: 536
bank: 515
fund: 484
business: 331
transaction: 291
million: 277
country: 274
company: 267
transfer: 266


## Extra features

In [94]:
# We add to the original dataframe two additional indicators (money symbols and suspicious words).
money_simbol_list = "|".join(["euro","dollar","pound","€",r"\$"])
suspicious_words = "|".join(["free","cheap","sex","money","account","bank","fund","transfer","transaction","win","deposit","password"])

data_train['money_mark'] = data_train['preprocessed_text'].str.contains(money_simbol_list)*1
data_train['suspicious_words'] = data_train['preprocessed_text'].str.contains(suspicious_words)*1
data_train['text_len'] = data_train['text'].apply(lambda x: len(x)) 

data_val['money_mark'] = data_val['preprocessed_text'].str.contains(money_simbol_list)*1
data_val['suspicious_words'] = data_val['preprocessed_text'].str.contains(suspicious_words)*1
data_val['text_len'] = data_val['preprocessed_text'].apply(lambda x: len(x)) 

data_train.head()

,text,label,preprocessed_text,money_mark,suspicious_words,text_len
381,Mills Cheryl D <MillsCD@state.gov>Sunday Janua...,0,mill cheryl sunday january pmtravel scheduleca...,1,0,157
428,H <hrod17@clintonemail.com >Saturday January 2...,0,saturday january pmsbwhoeopre fyi foreign nati...,1,0,207
849,"DEAR, MY NAME IS MR MR Ken Edward,A former...",1,dear name mr mr ken edwarda former government ...,1,1,1310
252,"Dear Sir, I am Engr. Victor Chigoziem with the...",1,dear sir engr victor chigoziem engineering sto...,1,1,3491
380,"Hello,This is Dr.Clive Whittaker. I work for F...",1,hellothis drclive whittaker work fidelity inve...,1,1,1327


## How would work the Bag of Words with Count Vectorizer concept?

In [96]:

# Separate ham (y=0) and spam (y=1) messages based on labels
ham_messages = data_train[data_train["label"] == 0]["preprocessed_text"]
spam_messages = data_train[data_train["label"] == 1]["preprocessed_text"]

# Initialize CountVectorizer
vectorizer = CountVectorizer()

# Fit and transform ham messages
X_ham = vectorizer.fit_transform(ham_messages)
ham_word_counts = X_ham.toarray().sum(axis=0)
ham_word_freq = dict(zip(vectorizer.get_feature_names_out(), ham_word_counts))

# Fit and transform spam messages
X_spam = vectorizer.fit_transform(spam_messages)
spam_word_counts = X_spam.toarray().sum(axis=0)
spam_word_freq = dict(zip(vectorizer.get_feature_names_out(), spam_word_counts))

# Get top 10 words
top_ham_words = sorted(ham_word_freq.items(), key=lambda x: x[1], reverse=True)[:10]
top_spam_words = sorted(spam_word_freq.items(), key=lambda x: x[1], reverse=True)[:10]

print("Top 10 words in ham messages:")
for word, freq in top_ham_words:
    print(f"{word}: {freq}")

print("\nTop 10 words in spam messages:")
for word, freq in top_spam_words:
    print(f"{word}: {freq}")

Top 10 words in ham messages:
president: 83
would: 80
percent: 76
call: 66
state: 65
american: 60
work: 58
obama: 57
pm: 57
mr: 56

Top 10 words in spam messages:
money: 594
account: 536
bank: 515
fund: 484
business: 331
transaction: 291
million: 277
country: 274
company: 267
transfer: 266


## TF-IDF

- Load the vectorizer

- Vectorize all dataset

- print the shape of the vetorized dataset

In [98]:
# Fit CountVectorizer on the entire dataset so it includes all words
X_train_counts = vectorizer.fit_transform(data_train['preprocessed_text'])
X_val_counts = vectorizer.transform(data_val['preprocessed_text'])

# Display feature names (words)
feature_names = vectorizer.get_feature_names_out()

# Initialize TfidfTransformer
tfidf_transformer = TfidfTransformer()

# Fit and transform the count matrix to get the TF-IDF matrix for training data
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

# Transform the count matrix to get the TF-IDF matrix for validation data
X_val_tfidf = tfidf_transformer.transform(X_val_counts)

## And the Train a Classifier?

In [100]:
# Initialize the Multinomial Naive Bayes classifier
nb_classifier = MultinomialNB()

# Fit the classifier on the training data
nb_classifier.fit(X_train_tfidf, data_train['label'])

# Predict on the validation data
y_pred = nb_classifier.predict(X_val_tfidf)

In [101]:
# Compute accuracy
accuracy = accuracy_score(data_val['label'], y_pred)

# Print accuracy
print(f"Accuracy: {accuracy * 100:.2f}%")

# Print classification report
print("Classification Report:")
print(classification_report(data_val['label'], y_pred))

Accuracy: 92.00%
Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.89      0.92       167
           1       0.87      0.96      0.91       133

    accuracy                           0.92       300
   macro avg       0.92      0.92      0.92       300
weighted avg       0.92      0.92      0.92       300



### Extra Task - Implement a SPAM/HAM classifier

https://www.kaggle.com/t/b384e34013d54d238490103bc3c360ce

The classifier can not be changed!!! It must be the MultinimialNB with default parameters!

Your task is to **find the most relevant features**.

For example, you can test the following options and check which of them performs better:
- Using "Bag of Words" only
- Using "TF-IDF" only
- Bag of Words + extra flags (money_mark, suspicious_words, text_len)
- TF-IDF + extra flags


You can work with teams of two persons (recommended).

In [104]:
# Initialize CountVectorizer and TfidfTransformer
vectorizer = CountVectorizer()
tfidf_transformer = TfidfTransformer()

# Fit and transform preprocessed text to get the TF-IDF matrix for training data
X_train_counts = vectorizer.fit_transform(data_train['preprocessed_text'])
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

# Transform preprocessed text to get the TF-IDF matrix for validation data
X_val_counts = vectorizer.transform(data_val['preprocessed_text'])
X_val_tfidf = tfidf_transformer.transform(X_val_counts)

# Convert TF-IDF sparse matrices to dense arrays
X_train_tfidf_dense = X_train_tfidf.toarray()
X_val_tfidf_dense = X_val_tfidf.toarray()

# Combine TF-IDF with additional features
additional_train_features = data_train[['money_mark', 'suspicious_words', 'text_len']].values
additional_val_features = data_val[['money_mark', 'suspicious_words', 'text_len']].values

X_train_combined = np.hstack((X_train_tfidf_dense, additional_train_features))
X_val_combined = np.hstack((X_val_tfidf_dense, additional_val_features))

In [105]:
# Initialize the Multinomial Naive Bayes classifier
nb_classifier = MultinomialNB()

# Fit the classifier on the training data
nb_classifier.fit(X_train_combined, data_train['label'])

# Predict on the validation data
y_pred = nb_classifier.predict(X_val_combined)

# Compute accuracy
accuracy = accuracy_score(data_val['label'], y_pred)

# Print accuracy
print(f"Accuracy: {accuracy * 100:.2f}%")

# Print classification report
print("Classification Report:")
print(classification_report(data_val['label'], y_pred))

Accuracy: 84.67%
Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.75      0.84       167
           1       0.75      0.97      0.85       133

    accuracy                           0.85       300
   macro avg       0.86      0.86      0.85       300
weighted avg       0.87      0.85      0.85       300



In [106]:
# Examine feature log probabilities
feature_names = np.array(vectorizer.get_feature_names_out().tolist() + ['money_mark', 'suspicious_words', 'text_len'])
log_probs = nb_classifier.feature_log_prob_

# Summarize feature importance
for i in range(log_probs.shape[0]):
    print(f"\nClass {i} feature log probabilities:")
    feature_importance = sorted(zip(log_probs[i], feature_names), reverse=True)
    for log_prob, feature in feature_importance[:10]:  # Top 10 features
        print(f"{feature}: {log_prob}")


Class 0 feature log probabilities:
text_len: -0.09084081231364394
money_mark: -6.206659727025651
suspicious_words: -8.743934362330968
fyi: -8.776707033549528
ok: -9.720671901090999
call: -9.731880829148091
yes: -9.765638864979
talk: -10.244434735572518
pls: -10.283244496374596
today: -10.391143771465426

Class 1 feature log probabilities:
text_len: -0.022636075374594
money_mark: -7.905651905080719
suspicious_words: -8.058727893778212
money: -10.767953992086236
account: -10.790136653681838
bank: -10.79232362567065
fund: -10.89003179122844
business: -11.22252308271903
transaction: -11.265598039317016
transfer: -11.297527225871708


In [111]:
# Building new classifier without text_length

# Initialize CountVectorizer and TfidfTransformer
vectorizer = CountVectorizer()
tfidf_transformer = TfidfTransformer()

# Combine TF-IDF with additional features
additional_train_features = data_train[['money_mark', 'suspicious_words']].values
additional_val_features = data_val[['money_mark', 'suspicious_words']].values

X_train_combined = np.hstack((X_train_tfidf_dense, additional_train_features))
X_val_combined = np.hstack((X_val_tfidf_dense, additional_val_features))

# Initialize the Multinomial Naive Bayes classifier
nb_classifier = MultinomialNB()

# Fit the classifier on the training data
nb_classifier.fit(X_train_combined, data_train['label'])

# Predict on the validation data
y_pred = nb_classifier.predict(X_val_combined)

# Compute accuracy
accuracy = accuracy_score(data_val['label'], y_pred)

# Print accuracy
print(f"Accuracy: {accuracy * 100:.2f}%")

# Print classification report
print("Classification Report:")
print(classification_report(data_val['label'], y_pred))


Accuracy: 94.00%
Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.92      0.94       167
           1       0.90      0.97      0.93       133

    accuracy                           0.94       300
   macro avg       0.94      0.94      0.94       300
weighted avg       0.94      0.94      0.94       300

